## Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install python-terrier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/119.5 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.9/347.9 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import pyterrier as pt
if not pt.started():
  pt.init()

<ipython-input-3-c43b832924c1>:2: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():


terrier-assemblies 5.10 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done


Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.10 (build: craigm 2024-08-22 17:33), helper_version=0.0.8]
<ipython-input-3-c43b832924c1>:3: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()


In [4]:
import pandas as pd

## Import data

In [15]:
df = pd.read_csv("/content/drive/MyDrive/INFO-376/pokemon_full.csv")

In [16]:
df['docno'] = df['docno'].astype(str)

In [17]:
df.head()

,docno,name,dexno,url,img,type,description,text
0,1,Bulbasaur,#0001,https://bulbapedia.bulbagarden.net/wiki/Bulbas...,https://img.pokemondb.net/sprites/home/normal/...,"Grass, Poison",A strange seed was planted on its back at birt...,"Bulbasaur Pokédex: stats, moves, evolution & l..."
1,2,Ivysaur,#0002,https://bulbapedia.bulbagarden.net/wiki/Ivysau...,https://img.pokemondb.net/sprites/home/normal/...,"Grass, Poison","When the bulb on its back grows large, it appe...","Ivysaur Pokédex: stats, moves, evolution & loc..."
2,3,Venusaur,#0003,https://bulbapedia.bulbagarden.net/wiki/Venusa...,https://img.pokemondb.net/sprites/home/normal/...,"Grass, Poison",The plant blooms when it is absorbing solar en...,"Venusaur Pokédex: stats, moves, evolution & lo..."
3,4,Charmander,#0004,https://bulbapedia.bulbagarden.net/wiki/Charma...,https://img.pokemondb.net/sprites/home/normal/...,Fire,"Obviously prefers hot places. When it rains, s...","Charmander Pokédex: stats, moves, evolution & ..."
4,5,Charmeleon,#0005,https://bulbapedia.bulbagarden.net/wiki/Charme...,https://img.pokemondb.net/sprites/home/normal/...,Fire,"When it swings its burning tail, it elevates t...","Charmeleon Pokédex: stats, moves, evolution & ..."


## Stopword Removal and Stemming

In [8]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# from nltk.stem import WordNetLemmatizer
import nltk
import string

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
# nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

Stemming is not used because we do not want pokemon names to be cropped.

In [18]:
stop_words = set(stopwords.words('english'))
custom_stop_words = {"pokémon", "pok", "mon", "type", "game", "pokédex", "stats", "moves"}
stop_words.update(custom_stop_words)

def remove_stopwords_stemming(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word not in stop_words]
    # tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

df['text'] = df['text'].apply(remove_stopwords_stemming)
df['text'] = df['text'].apply(lambda x: ' '.join(x))

In [19]:
df.head()

,docno,name,dexno,url,img,type,description,text
0,1,Bulbasaur,#0001,https://bulbapedia.bulbagarden.net/wiki/Bulbas...,https://img.pokemondb.net/sprites/home/normal/...,"Grass, Poison",A strange seed was planted on its back at birt...,bulbasaur evolution locations database skip ma...
1,2,Ivysaur,#0002,https://bulbapedia.bulbagarden.net/wiki/Ivysau...,https://img.pokemondb.net/sprites/home/normal/...,"Grass, Poison","When the bulb on its back grows large, it appe...",ivysaur evolution locations database skip main...
2,3,Venusaur,#0003,https://bulbapedia.bulbagarden.net/wiki/Venusa...,https://img.pokemondb.net/sprites/home/normal/...,"Grass, Poison",The plant blooms when it is absorbing solar en...,venusaur evolution locations database skip mai...
3,4,Charmander,#0004,https://bulbapedia.bulbagarden.net/wiki/Charma...,https://img.pokemondb.net/sprites/home/normal/...,Fire,"Obviously prefers hot places. When it rains, s...",charmander evolution locations database skip m...
4,5,Charmeleon,#0005,https://bulbapedia.bulbagarden.net/wiki/Charme...,https://img.pokemondb.net/sprites/home/normal/...,Fire,"When it swings its burning tail, it elevates t...",charmeleon evolution locations database skip m...


## Indexing

In [21]:
pd_indexer = pt.IterDictIndexer('/content/drive/MyDrive/INFO-376/test2_index',
                                meta={'docno': 20, 'text': 10000},
                                meta_tags={'docno': 'identifier', 'name': 'text', 'dexno': 'text', 'url': 'string', 'img': 'string',
                                           'type': 'text', 'description': 'text', 'text': 'text'},
                                  stemmer=None, stopwords=None, tokenizer='whitespace')

indexref = pd_indexer.index(df.to_dict(orient='records'))

index = pt.IndexFactory.of(indexref)
print (index.getCollectionStatistics().toString())

Number of documents: 1025
Number of terms: 22934
Number of postings: 552481
Number of fields: 1
Number of tokens: 1397012
Field names: [text]
Positions:   false



In [22]:
data = []

for term, le in index.getLexicon():
    new_row = {'term': term, 'frequency': le.getFrequency()}
    data.append(new_row)

terms = pd.DataFrame(data)
terms = terms.sort_values(by='frequency', ascending=False)
terms.head(20)

,term,frequency
1013,100,60716
14502,normal,39314
2142,90,17407
1330,2,11921
16377,psychic,11894
2077,80,11349
1011,1,10499
14007,move,9841
1926,60,9449
16079,power,9389


In [14]:
iter = index.get_corpus_iter(return_toks=True)
next(iter)

{'toks': {'0': 2,
  '24': 2,
  'nourishment': 4,
  'content': 2,
  'notable': 1,
  'based': 4,
  'three': 1,
  'upon': 1,
  'lizard': 1,
  'hp': 2,
  'fire': 2,
  'isanghaessi': 1,
  'yellow': 3,
  'cavern': 3,
  'ele': 1,
  'brilliant': 7,
  'diamondpearl': 1,
  'info': 1,
  'know': 1,
  'red': 6,
  'strength': 1,
  'cat': 6,
  'goldsilvercrystal': 1,
  'blue': 5,
  '87': 1,
  'special': 1,
  '53': 1,
  'go': 11,
  '54': 1,
  'anyone': 1,
  'find': 2,
  'move': 7,
  '6': 5,
  'natures': 1,
  'maximum': 1,
  'squirtle': 5,
  'napping': 2,
  'additional': 1,
  'fairy': 2,
  'evolves': 1,
  'ruby': 6,
  'pikachulets': 1,
  'picnics': 1,
  'seeds': 1,
  'locations': 3,
  '103': 1,
  '252': 1,
  'birth': 10,
  'saur': 1,
  'ev': 1,
  '58': 1,
  'seed': 36,
  '66': 1,
  '8': 4,
  'good': 1,
  'heartgoldsoulsilver': 1,
  'strange': 4,
  'suggestion': 1,
  '0164': 1,
  'hidden': 1,
  'groups': 2,
  'gro': 1,
  'knock': 1,
  'morsel': 2,
  'frogtoad': 1,
  'attack': 1,
  'better': 1,
  'tackle

# Retrieval

In [43]:
queries = pd.DataFrame([["q1","bulbasaur"], ["q2","green bulb"], ["q3", "bubasaur"], ["q4", "green plant with bulb"]], columns=["qid","query"])

In [44]:
queries

,qid,query
0,q1,bulbasaur
1,q2,green bulb
2,q3,bubasaur
3,q4,green plant with bulb


In [45]:
index = pt.IndexFactory.of("/content/drive/MyDrive/INFO-376/poke_index/data.properties")
tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
lm = pt.BatchRetrieve(index, wmodel="Hiemstra_LM")

tf_idf_results = tf_idf.transform(queries)
bm25_results = bm25.transform(queries)
lm_results = lm.transform(queries)

tf_idf_top10 = tf_idf_results.groupby("qid").head(10)
bm25_top10 = bm25_results.groupby("qid").head(10)
lm_top10 = lm_results.groupby("qid").head(10)

<ipython-input-45-6b34b940b180>:2: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")
<ipython-input-45-6b34b940b180>:3: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25 = pt.BatchRetrieve(index, wmodel="BM25")
<ipython-input-45-6b34b940b180>:4: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  lm = pt.BatchRetrieve(index, wmodel="Hiemstra_LM")


In [46]:
tf_idf_top10

,qid,docid,docno,rank,score,query
0,q1,0,1,0,8.933193,bulbasaur
1,q1,1,2,1,8.112356,bulbasaur
2,q1,2,3,2,7.746547,bulbasaur
3,q1,3,4,3,6.858287,bulbasaur
4,q1,6,7,4,6.411948,bulbasaur
5,q2,0,1,0,11.256384,green bulb
6,q2,1,2,1,8.364326,green bulb
7,q2,547,548,2,6.118311,green bulb
8,q2,937,938,3,5.113431,green bulb
9,q2,780,781,4,4.594541,green bulb


In [36]:
qrels = pd.DataFrame([["q1", "1", 1],
                     ["q2", "1", 1],
                     ["q2", "2", 1],
                     ["q2", "3", 1],
                     ["q3", "1", 1],
                     ["q4", "1", 1],
                     ["q4", "2", 1],
                     ["q4", "3", 1],], columns=["qid", "docno", "label"])

In [37]:
qrels

,qid,docno,label
0,q1,1,1
1,q2,1,1
2,q2,2,1
3,q2,3,1
4,q3,1,1
5,q4,1,1
6,q4,2,1
7,q4,3,1


In [38]:
pt.Experiment (
    [tf_idf, bm25, lm],
    queries,
    qrels,
    eval_metrics=["map", "Rprec", "recip_rank"]
)

,name,map,Rprec,recip_rank
0,TerrierRetr(TF_IDF),0.683333,0.583333,0.75
1,TerrierRetr(BM25),0.683333,0.583333,0.75
2,TerrierRetr(Hiemstra_LM),0.687500,0.583333,0.75


# Meta Tags Retrieval

In [ ]:
doc_id = 0 # Example doc
                                        # options: name, dexno, url, img, type, description, text, docno
poke_name = index.getMetaIndex().getItem("name", doc_id)
poke_description = index.getMetaIndex().getItem("description", doc_id)

print(poke_name)
print()
print(poke_description) # one sentence description